In [1]:
import os
import time
import requests
from dotenv import load_dotenv
# Importaciones de Pinecone
from pinecone import Pinecone, ServerlessSpec
# Importaciones de Langchain
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter # Importación corregida
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
load_dotenv()

/home/josefc/dev/test_to_drop/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
ARCHIVOS_URL = [
    "https://preguntapdf.s3.eu-south-2.amazonaws.com/la-rosaleda_v3.pdf",
    "https://preguntapdf.s3.eu-south-2.amazonaws.com/GuiaViajeMexicoDF.pdf"
]

NOMBRE_ARCHIVO_PDF_LOCAL = ARCHIVOS_URL[0].split("/")[-1]
# "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" tiene una dimensión de 384.
EMBEDDING_DIMENSION = 384 # Corregido para el modelo especificado
PINECONE_NAMESPACE = None
# Nombre del modelo de embeddings de HuggingFace
MODELO_EMBEDDINGS = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

if not PINECONE_API_KEY or not PINECONE_INDEX_NAME:
    raise ValueError("Las variables de entorno PINECONE_API_KEY y PINECONE_INDEX_NAME deben estar configuradas.")

print(f"Usando índice de Pinecone: {PINECONE_INDEX_NAME}")
print(f"Dimensión de embeddings: {EMBEDDING_DIMENSION}")
print(f"Modelo de embeddings: {MODELO_EMBEDDINGS}")

Usando índice de Pinecone: myindex2
Dimensión de embeddings: 384
Modelo de embeddings: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


In [3]:
for url in ARCHIVOS_URL:
    nombre_archivo = url.split("/")[-1]
    if not os.path.exists(nombre_archivo): # Descargar solo si no existe
        try:
            print(f"Descargando {nombre_archivo} desde {url}...")
            doc_to_download = requests.get(url, timeout=30) # Añadido timeout
            doc_to_download.raise_for_status() # Lanza un error para respuestas HTTP malas (4xx o 5xx)
            with open(nombre_archivo, "wb") as pdf_file:
                pdf_file.write(doc_to_download.content)
            print(f"{nombre_archivo} descargado y guardado.")
        except requests.exceptions.RequestException as e:
            print(f"Error al descargar {nombre_archivo}: {e}")
    else:
        print(f"El archivo {nombre_archivo} ya existe localmente.")

Descargando la-rosaleda_v3.pdf desde https://preguntapdf.s3.eu-south-2.amazonaws.com/la-rosaleda_v3.pdf...
la-rosaleda_v3.pdf descargado y guardado.
Descargando GuiaViajeMexicoDF.pdf desde https://preguntapdf.s3.eu-south-2.amazonaws.com/GuiaViajeMexicoDF.pdf...
GuiaViajeMexicoDF.pdf descargado y guardado.


In [4]:
try:
    loader = PyPDFLoader(NOMBRE_ARCHIVO_PDF_LOCAL)
    documentos_cargados = loader.load() # .load() devuelve una lista de Documentos
    print(f"PDF cargado. Número de páginas/documentos iniciales: {len(documentos_cargados)}")

    # Función para dividir los documentos en chunks más pequeños
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,      # Tamaño deseado para cada chunk
        chunk_overlap=100,   # Superposición entre chunks para mantener contexto
        length_function=len  # Función para medir la longitud (caracteres por defecto)
    )
    chunks = text_splitter.split_documents(documentos_cargados)
    print(f"Documentos divididos en {len(chunks)} chunks.")
    if chunks:
        print(f"Ejemplo de chunk: '{chunks[0].page_content[:100]}...'")

except Exception as e:
    print(f"Error al cargar o dividir el PDF: {e}")
    raise

PDF cargado. Número de páginas/documentos iniciales: 2
Documentos divididos en 7 chunks.
Ejemplo de chunk: 'Bienvenidos a La Rosaleda. 
El recinto blanquiazul, con más de 75 años de  historia, es la casa del ...'


In [5]:
try:
    embeddings = HuggingFaceEmbeddings(model_name=MODELO_EMBEDDINGS)
    print(f"Modelo de embeddings '{MODELO_EMBEDDINGS}' inicializado.")
    # Prueba rápida del modelo de embeddings (opcional)
    # test_embedding = embeddings.embed_query("Esto es una prueba.")
    # print(f"Dimensión del embedding de prueba: {len(test_embedding)}")
except Exception as e:
    print(f"Error al inicializar el modelo de embeddings: {e}")
    raise

/tmp/ipykernel_251776/2978058678.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=MODELO_EMBEDDINGS)
/home/josefc/dev/test_to_drop/env/lib/python3.12/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Modelo de embeddings 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2' inicializado.


In [6]:
print("Inicializando cliente Pinecone...")
try:
    pc = Pinecone(api_key=PINECONE_API_KEY)
    print("Cliente Pinecone inicializado.")
except Exception as e:
    print(f"Error inicializando cliente Pinecone: {e}")
    raise

print(f"Comprobando/Creando índice serverless '{PINECONE_INDEX_NAME}'...")
try:
    index_list = pc.list_indexes()
    existing_index_names = [index_info.name for index_info in index_list.indexes]

    if PINECONE_INDEX_NAME not in existing_index_names:
        print(f"Índice '{PINECONE_INDEX_NAME}' no encontrado. Creando nuevo índice sin servidor...")
        pc.create_index(
            name=PINECONE_INDEX_NAME,
            dimension=EMBEDDING_DIMENSION,
            metric="cosine", # Métrica común para similitud semántica
            spec=ServerlessSpec(
                cloud="aws",        # O "gcp", "azure" según tu proveedor
                region="us-east-1"  # Cambia esto a tu región de Pinecone Serverless (ej. "us-west-2" para AWS)
                                    # os.getenv("PINECONE_LOC") podría usarse aquí si define la región.
            )
        )
        print(f"Creación del índice '{PINECONE_INDEX_NAME}' iniciada. Esperando a que esté listo...")
        while not pc.describe_index(PINECONE_INDEX_NAME).status['ready']:
            print("Esperando que el índice esté listo...")
            time.sleep(5)
        print(f"Índice '{PINECONE_INDEX_NAME}' está listo.")
    else:
        print(f"Índice '{PINECONE_INDEX_NAME}' ya existe.")

    # Obtener el host para el índice e instanciar un cliente de Índice específico
    index_description = pc.describe_index(PINECONE_INDEX_NAME)
    index_host = index_description.host
    print(f"Host para el índice '{PINECONE_INDEX_NAME}': {index_host}")

    pinecone_index_object = pc.Index(host=index_host)
    print(f"Cliente de índice Pinecone para el host '{index_host}' inicializado.")

except Exception as e:
    print(f"Error durante la creación/comprobación del índice o al obtener el cliente del índice para '{PINECONE_INDEX_NAME}': {e}")
    raise

Inicializando cliente Pinecone...
Cliente Pinecone inicializado.
Comprobando/Creando índice serverless 'myindex2'...
Índice 'myindex2' ya existe.
Host para el índice 'myindex2': myindex2-2sr4jlm.svc.aped-4627-b74a.pinecone.io
Cliente de índice Pinecone para el host 'myindex2-2sr4jlm.svc.aped-4627-b74a.pinecone.io' inicializado.


In [7]:
if not chunks:
    print("No hay chunks para cargar. Saltando la carga a Pinecone.")
else:
    try:
        vectorstore = PineconeVectorStore(
            index=pinecone_index_object,      # Objeto de índice Pinecone específico
            embedding=embeddings,             # Función de embedding de Langchain
            text_key="text",                  # Clave estándar para el contenido del texto en metadatos de Pinecone
            namespace=PINECONE_NAMESPACE      # Namespace (opcional)
        )
        print("PineconeVectorStore (Langchain) inicializado.")

        # Añadir los chunks (documentos) al almacén de vectores en Pinecone
        print(f"Añadiendo {len(chunks)} fragmentos de documentos al índice '{PINECONE_INDEX_NAME}' (namespace: {PINECONE_NAMESPACE or 'default'})...")
        vectorstore.add_documents(documents=chunks)
        print("Documentos añadidos a Pinecone mediante PineconeVectorStore.")

    except Exception as e:
        print(f"Error durante las operaciones de PineconeVectorStore (inicialización o adición de documentos): {e}")
        raise

PineconeVectorStore (Langchain) inicializado.
Añadiendo 7 fragmentos de documentos al índice 'myindex2' (namespace: default)...
Documentos añadidos a Pinecone mediante PineconeVectorStore.


In [8]:

if not chunks: # No se puede consultar si no se cargaron chunks
    print("No se cargaron chunks, saltando la consulta de ejemplo.")
else:
    try:
        query_text = "¿Cuál es el tema principal de los documentos?" # Tu texto de consulta
        print(f"Realizando búsqueda de similitud para: '{query_text}'")

        search_results = vectorstore.similarity_search(
            query=query_text,
            k=3, # Número de resultados a devolver
            namespace=PINECONE_NAMESPACE # Especifica el namespace si se usa
        )
        print("\n")
        print(f"Top {len(search_results)} resultados de búsqueda de Langchain:")
        for i, doc_result in enumerate(search_results):
            print(f"--- Resultado {i+1} ---")
            print(f"Contenido: {doc_result.page_content[:250]}...") # Imprime los primeros 250 caracteres
            if doc_result.metadata:
                print(f"Metadatos: {doc_result.metadata}")
            print("-" * 20)

    except Exception as e:
        print(f"Error durante la búsqueda de similitud de Langchain: {e}")

Realizando búsqueda de similitud para: '¿Cuál es el tema principal de los documentos?'


Top 3 resultados de búsqueda de Langchain:
--- Resultado 1 ---
Contenido: a hacerlo) cualquier descripción, anotación, imagen, video, audio, dato, estadística u 
otra f orma de repro ducción d el even to, excep to para u so persona l y privado, cons tituye ndo 
el incumplimiento de cualquiera de dichas obligaciones igualme...
Metadatos: {'page': 1.0, 'source': 'la-rosaleda_v3.pdf'}
--------------------
--- Resultado 2 ---
Contenido: a hacerlo) cualquier descripción, anotación, imagen, video, audio, dato, estadística u 
otra f orma de repro ducción d el even to, excep to para u so persona l y privado, cons tituye ndo 
el incumplimiento de cualquiera de dichas obligaciones igualme...
Metadatos: {'page': 1.0, 'source': 'la-rosaleda_v3.pdf'}
--------------------
--- Resultado 3 ---
Contenido: a hacerlo) cualquier descripción, anotación, imagen, video, audio, dato, estadística u 
otra f orma de repro du